# 2. Symbolic Pursuit vs LIME

In this notebook, we build a black-box MLP model for the UCI *wine quality* dataset *(Cortez et al. 2009)*.
We then build a symbolic model for this black-box and we compare our model to a *LIME* explainer *(Ribeiro, Singh, and Guestrin 2016)*. Let us begin with the useful imports.


In [ ]:
from symbolic_pursuit.models import SymbolicRegressor
from experiments.train_model import train_model
from datasets.data_loader_UCI import mixup, data_loader
from sympy.printing.latex import latex
import sklearn
import lime 
import lime.lime_tabular
import numpy as np

As before, we split the dataset into a training and a test set. We use the mixup technique to produce a training set for the faithful model *(Zhang et al. 2018)*. 

In [ ]:
X_train, y_train, X_test, y_test = data_loader(dataset_name='wine-quality-red')
X_random = mixup(X_train)

We train the MLP blackbox.

In [ ]:
model = train_model(X_train, y_train, black_box="MLP")

We train the symbolic model.

In [ ]:
symbolic_model = SymbolicRegressor()
symbolic_model.fit(model.predict, X_random)

We train a *LIME* explainer for this regression problem.

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_train, 
                                                   feature_names=["X"+str(k) for k in range(len(X_train[0]))], 
                                                   class_names=['quality'], 
                                                   verbose=True,
                                                   mode='regression')

We ask to the LIME interpeter an explanation for a prediction in the test set.

In [ ]:
i = 5
exp = explainer.explain_instance(X_test[i], model.predict, num_features=5)
exp.show_in_notebook(show_table=True)

It seems like $X_{10}$ is the most important feature for this prediction. A linear expansion of our symbolic model around this test point should give a similar result. Let us produce a first order Taylor expansion of our faithful model around this point.

In [ ]:
poly1 = symbolic_model.get_taylor(X_test[i], approx_order=1)
poly1

We can easily get a $\LaTeX$ expression.

In [ ]:
latex(poly1)

We observe that, indeed, $X_{10}$ has the highest weight in this local linear model. Also note that, as suggested by *LIME*, $X_9$ also has an important weight in this polynomial. However, the agreement is not perfect since $X_2$ has a small weight in our local model. One big advantage of our *Symbolic Pursuit* scheme is that we can actually choose the order of the local model obtained by performing a Taylor expansion in a post-hoc analysis. For instance, we can get an idea of the interactions by going one order above.

In [ ]:
poly2 = symbolic_model.get_taylor(X_test[i], approx_order=2)
poly2

We also extract the $\LaTeX$ expression easily.

In [ ]:
latex(poly2)

We can develop these square to have a less compact expression where we can look at the individual interactions

In [ ]:
from sympy import expand
expand(poly2)

By having a quick look at this expression, we note that the most significant interaction is between $X_{6}$ and $X_{10}$ followed closely by the interaction between $X_{1}$ and $X_{10}$.

## References

Cortez, Paulo, António Cerdeira, Fernando Almeida, Telmo Matos, and José Reis. 2009. “Modeling Wine Preferences by Data Mining from Physicochemical Properties.” Decision Support Systems 47 (4): 547–53. doi:10.1016/j.dss.2009.05.016.

Ribeiro, Marco Tulio, Sameer Singh, and Carlos Guestrin. 2016. “‘Why Should I Trust You?’: Explaining the Predictions of Any Classifier.” arXiv:1602.04938 [cs, Stat], August. http://arxiv.org/abs/1602.04938.

Zhang, Hongyi, Moustapha Cisse, Yann N. Dauphin, and David Lopez-Paz. 2018. “Mixup: Beyond Empirical Risk Minimization.” arXiv:1710.09412 [cs, Stat], April. http://arxiv.org/abs/1710.09412.